## Training default

In [ ]:
from ultralytics import YOLO
import torch
import os

"""
BEVDetNet params:
epochs: 50, 60, 80, 120
batch_size: 8, 16, 24, 32, 48
imgsz: 640x640x3
optimizer: Adam
loss weights: 0.95, 0.98, 1
"""

# Set expandable_segments to reduce memory fragmentation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Check for GPU use
print('Use GPU:', torch.cuda.is_available())

if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("GPU cache cleared.\n")

# Build a new model from scratch and check model specific values
print('--Check YOLO intern values for custom purposes--')
model = YOLO('yolov8s-obb.yaml', 'obb', verbose=False) # n/s/m/l/x 
# Print layers, parameters, gradients, GFLOPS (computation depends on https://github.com/ultralytics/ultralytics/issues/17547#issuecomment-2481925742)
# https://github.com/ultralytics/ultralytics/issues/14749
print(model.info(detailed=False, verbose=True))

# https://community.ultralytics.com/t/about-yolo-configuration-file-yaml/300
results = model.train(data='/home/rlab10/ultralytics_yolov8-obb_ob_kitti/ultralytics/cfg/datasets/kitti_bev.yaml', epochs=20,
                      time=None, patience=10, batch=8, imgsz=640, save=True, save_period=2, cache=False,
                      device='cuda', workers=4, project='kitti_bev_yolo', name='run_', exist_ok=False,
                      pretrained=False, optimizer='Adam', seed=0, deterministic=False, single_cls=False,
                      classes=None, rect=False, cos_lr=False, close_mosaic=0, resume=True, amp=False,
                      fraction=1.0, profile=False, freeze=None, lr0=0.01, lrf=0.01,  momentum=0.937,
                      weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, 
                      box=7.5, cls=0.5, dfl=1.5, pose=0.0, kobj=0.0, nbs=64, overlap_mask=False, mask_ratio=0,
                      dropout=0.0, val=True, plots=False,
                      
                      hsv_h=0.0, hsv_s=0.0, hsv_v=0.0, degrees=0.0, translate=0.0, scale=0.0, shear=0.0, perspective=0.0,
                      flipud=0.0, fliplr=0.0, bgr=0.0, mosaic=0.0, mixup=0.0, copy_paste=0.0, copy_paste_mode='flip', auto_augment=None,
                      erasing=0.0, crop_fraction=0.0)

# For training, by default you'll get the final validation confusion matrix with
#results.confusion_matrix.matrix

## Training with WandB Integration

In [ ]:
import wandb
from ultralytics import YOLO
import torch
import os

"""
BEVDetNet params:
epochs: 50, 60, 80, 120
batch_size: 8, 16, 24, 32, 48
imgsz: 640x640x3
optimizer: Adam
loss weights: 0.95, 0.98, 1
"""

# Set expandable_segments to reduce memory fragmentation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Check for GPU use
print('Use GPU:', torch.cuda.is_available())

if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("GPU cache cleared.\n")

wandb.login()

# Define sweep configuration
sweep_config = {
    'method': 'bayes',
    'metric': {
        'name': 'train/cls_loss', # Metric to optimize 
        'goal': 'minimize'
    },
    'parameters': {
        'epochs': {
            'values': [50, 60, 80, 100, 120]
        },
        'batch': {
            'values': [4, 8, 16, 24, 32, 48]
        },
        'optimizer': {
            'values': ['SGD', 'Adam', 'AdamW', 'NAdam', 'RAdam', 'RMSProp', 'auto']
        },
        # Initial learning rate
        'lr0': {
            'distribution': 'log_uniform',
            'min': 1e-5,
            'max': 1e-1
        },
        # Final learning rate
        'lrf': {
            'distribution': 'log_uniform',
            'min': 1e-5,
            'max': 1e-1
        }, 
        # Factor for optimizer
        'momentum': {
            'distribution': 'uniform',
            'min': 0.6,
            'max': 0.98
        },
        # L2 regularization term
        'weight_decay': {
            'distribution': 'uniform',
            'min': 0.0001,
            'max': 0.0006
        },
        'warmup_epochs': {
            'distribution': 'uniform',
            'min': 0.0,
            'max': 5.0
        },
        'warmup_momentum':{
            'distribution': 'uniform',
            'min': 0.0,
            'max': 0.95
        },
        'warmup_bias_lr': {
            'distribution': 'uniform',
            'min': 0.1,
            'max': 0.6
        },
        'box': {
            'values': [6.0, 6.5, 7.0, 7.5, 8.0, 8.5] # Box loss weight
        },
        'cls': {
            'values': [0.3, 0.5, 0.7, 1.0, 1.2, 1.5] # Class loss weight
        },
        'dfl': {
            'values': [0.5, 1.0, 1.2, 1.5, 1.8, 2.0] # DFL loss weight
        },
        'nbs': {
            'values': [4, 8, 16, 24, 32, 48, 64] # nominal bt, normalization loss
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project='kitti_bev_yolo_wandb')

def train():
    try:
        # Initialize WandB run
        wandb.init(job_type='training')

        # Build a new model from scratch and check model specific values
        print('--Check YOLO intern values for custom purposes--')
        # Build a new model from scratch
        model = YOLO('yolov8s-obb.yaml', 'obb', verbose=False)

        # Train the model with hyperparameters from WandB
        results = model.train(data='/home/rlab10/ultralytics_yolov8-obb_ob_kitti/ultralytics/cfg/datasets/kitti_bev.yaml', epochs=wandb.config.epochs,
                            time=None, patience=10, batch=wandb.config.batch, imgsz=640, save=True, save_period=25, cache=True,
                            device='cuda', workers=2, project='kitti_bev_yolo_wandb', name=f'epo._{wandb.config.epochs}_bat._{wandb.config.batch}', 
                            exist_ok=False, pretrained=False, optimizer=wandb.config.optimizer, seed=0, deterministic=False, single_cls=False,
                            classes=None, rect=False, cos_lr=False, close_mosaic=0, resume=True, amp=False, fraction=1.0, profile=False, freeze=None, 
                            lr0=wandb.config.lr0, lrf=wandb.config.lrf,  momentum=wandb.config.momentum, weight_decay=wandb.config.weight_decay, 
                            warmup_epochs=wandb.config.warmup_epochs, warmup_momentum=wandb.config.warmup_momentum, warmup_bias_lr=wandb.config.warmup_bias_lr, 
                            box=wandb.config.box, cls=wandb.config.cls, dfl=wandb.config.dfl, pose=0.0, kobj=0.0, nbs=wandb.config.nbs, overlap_mask=False, 
                            mask_ratio=0, dropout=0.0, val=True, plots=False,
                            
                            hsv_h=0.0, hsv_s=0.0, hsv_v=0.0, degrees=0.0, translate=0.0, scale=0.0, shear=0.0, perspective=0.0,
                            flipud=0.0, fliplr=0.0, bgr=0.0, mosaic=0.0, mixup=0.0, copy_paste=0.0, copy_paste_mode='flip', auto_augment=None,
                            erasing=0.0, crop_fraction=0.0)
    finally:   
        wandb.finish()

wandb.agent(sweep_id=sweep_id, function=train, count=18)

# TODO: wenn die optimalen Parameter gefunden wurden, training mit plots=True

## Training with Ray Tune Integration

In [ ]:
from ray import tune
from ray.air.integrations.wandb import WandbLoggerCallback
from ultralytics import YOLO
import torch
import os

"""
BEVDetNet params:
epochs: 50, 60, 80, 120
batch_size: 8, 16, 24, 32, 48
imgsz: 640x640x3
optimizer: Adam
loss weights: 0.95, 0.98, 1
"""

# Set expandable_segments to reduce memory fragmentation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Check for GPU use
print('Use GPU:', torch.cuda.is_available())

if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("GPU cache cleared.\n")

tune_config = {
    "epochs": tune.choice([50, 60, 80, 100, 120]),
    "batch": tune.choice([4, 8, 16, 24, 32, 48]),
    "optimizer": tune.choice(["SGD", "Adam", "AdamW", "NAdam", "RAdam", "RMSProp", "auto"]),
    "lr0": tune.loguniform(1e-5, 1e-1),  # Initial learning rate, Ray Tune default: (1e-5, 1e-1)
    "lrf": tune.loguniform(1e-5, 1e-1),  # Final learning rate, Ray Tune default: (0.01, 1.0)
    "momentum": tune.uniform(0.6, 0.98), # Ray Tune default: (0.6, 0.98)
    "weight_decay": tune.uniform(0.0001, 0.0006), # Ray Tune default: (0.0, 0.001)
    "warmup_epochs": tune.uniform(0.0, 5.0), #  Ray Tune default: (0.0, 5.0)
    "warmup_momentum": tune.uniform(0.0, 0.95), # Ray Tune default: (0.0, 0.95)
    "warmup_bias_lr": tune.uniform(0.1, 0.6), 
    "box": tune.choice([6.0, 6.5, 7.0, 7.5, 8.0, 8.5]),  # Box loss weight Ray Tune default: (0.02, 0.2)
    "cls": tune.choice([0.3, 0.5, 0.7, 1.0, 1.2, 1.5]),  # Class loss weight Ray Tune default:  (0.2, 4.0)
    "dfl": tune.choice([0.5, 1.0, 1.2, 1.5, 1.8, 2.0]),  # DFL loss weight
    "nbs": tune.choice([4, 8, 16, 24, 32, 48, 64]),  # Nominal batch size
}

# Build a new model from scratch and check model specific values
print('--Check YOLO intern values for custom purposes--')
# Build a new model from scratch
model = YOLO('yolov8s-obb.yaml', 'obb', verbose=False)

# Train the model with hyperparameters from Ray Tune
result_grid = model.tune(
    data='/home/rlab10/ultralytics_yolov8-obb_ob_kitti/ultralytics/cfg/datasets/kitti_bev.yaml', use_ray=True, space=tune_config,
    grace_period=10, gpu_per_trial=1, iterations=10,
    # **train_args
    time=None, patience=10, imgsz=640, save=True, save_period=25, cache=True, device=0, workers=4, project='kitti_bev_yolo_rayt', 
    name='tune_experiment', exist_ok=False, pretrained=False, seed=0, deterministic=False, single_cls=False, classes=None, rect=False, 
    cos_lr=False, close_mosaic=0, resume=True, amp=False, fraction=1.0, profile=False, freeze=None,  pose=0.0, kobj=0.0,  overlap_mask=False, 
    mask_ratio=0, dropout=0.0, val=True, plots=False,
    # augmentation args                      
    hsv_h=0.0, hsv_s=0.0, hsv_v=0.0, degrees=0.0, translate=0.0, scale=0.0, shear=0.0, perspective=0.0,flipud=0.0, fliplr=0.0, bgr=0.0, mosaic=0.0, 
    mixup=0.0, copy_paste=0.0, copy_paste_mode='flip', auto_augment=None,erasing=0.0, crop_fraction=0.0)


## Validation

In [ ]:
from ultralytics import YOLO
import torch

"""
BEVDetNet params:
IoU: 0.5, 0.7
conf: 0.5, 0.7
Class: Car (only)
Difficulties: Easy, Moderate, Hard
"""

"""
Bird's Eye View Benchmark
=========================

The goal in the bird's eye view detection task is to train object detectors
for the classes 'Car', 'Pedestrian', and 'Cyclist' where the detectors must provide
BOTH the 2D 0-based bounding box in the image as well as the 3D bounding box
in bird's eye view and the detection score/confidence. This means that the 3D
bounding box does not have to include information on the height axis, i.e.
the height of the bounding box and the bounding box location along the height axis.
For example, when evaluating the bird's eye view benchmark only (without the
3D object detection benchmark), the height of the bounding box can be set to
a value equal to or smaller than zero. Similarly, the y-axis (camera) location of the
bounding box can be set to -1000 (note that an arbitrary negative value will
not work). As above, we note that the 2D bounding boxes are required to filter
objects larger than 25 pixel (height) and that - to avoid false positives - detections
not visible on the image plane should be filtered. As in all benchmarks, we do
not count 'Van' as false positives for 'Car' or 'Sitting Person' as false positive
for 'Pedestrian'. Evaluation criterion follows the above benchmarks using
a bird's eye view bounding box overlap.
"""

"""
from 8.10.2019 we compute the average precision not like in the PASCAL VOC protocol, but as follows:

sum = 0;
for (i=1; i<=40; i++)
  sum += vals[i];
average = sum/40.0;
"""

# Check for GPU use
print('Use GPU:', torch.cuda.is_available())

if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("GPU cache cleared.\n")

# Load my custom trained model
model = YOLO('path/to/best.pt') # validate TensorRT model here

# Validate the model (does this call bbox_iou)
metrics = model.val(data='/home/rlab10/ultralytics/ultralytics/cfg/datasets/kitti_bev.yaml', imgsz=640, 
                    batch=-1, save_json=True, save_hybrid=False, conf=0.5, iou=0.5, max_det=50, half=False,
                    device='cuda', dnn=False, plots=True, rect=False, split='val', project='kitti_bev_yolo', name='val_')

# BEVDetNet metrics (adapted to 40-point interpolation like KITTI)
print(metrics.box.ap50) # AP at IoU threshold of 0.5 for all classes
print(metrics.box.ap70) # AP at IoU threshold of 0.7 for all classes


# Access during validation
metrics.confusion_matrix.matrix





## Best model was saved at epoch 
https://github.com/ultralytics/ultralytics/issues/14137

In [ ]:
import pandas as pd

# Load the training log
results = pd.read_csv('path/to/results.csv')

# Strip spaces
results.columns = results.columns.str.strip()

# Calculate fitness
results["fitness"] = results["metrics/mAP50(B)"] * 0.1 + results["metrics/mAP50-95(B)"] * 0.9

# Find the epoch with the highest fitness
best_epoch = results['fitness'].idxmax() + 1

print(f"Best model was saved at epoch: {best_epoch}")